In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from rake_nltk import Rake

In [3]:
key = pd.read_csv('place_data.csv')

In [4]:
key.head()

,Unnamed: 0,이름,코드,태그
0,0,고른햇살,Num=a_nam_0002,#먹어야산다#분식#정대후문#포장#혼밥#가성비
1,1,영철버거,Num=a_nam_0041,#먹어야산다#이공계#경양식#버거#맥주#치킨#가성비#저렴#포장
2,2,어흥식당,Num=a_nam_829,#먹어야산다#양식#스테이크#정대후문#혼밥#고기#함박#가성비
3,3,언니네반점,Num=a_nam_0146,#먹어야산다#중식#정대후문#사천식#혼밥#가성비#저렴#포장#회식
4,4,특별식당,Num=a_nam_0319,#먹어야산다#일식#돈부리#카레#맥주#고로케#제기동#고대사거리#혼밥#감성


In [5]:
key = key[['이름', '코드', '태그']]
key.columns = ['name', 'code', 'tag']

In [6]:
print(key.shape)

(465, 3)


In [7]:
key.head()

,name,code,tag
0,고른햇살,Num=a_nam_0002,#먹어야산다#분식#정대후문#포장#혼밥#가성비
1,영철버거,Num=a_nam_0041,#먹어야산다#이공계#경양식#버거#맥주#치킨#가성비#저렴#포장
2,어흥식당,Num=a_nam_829,#먹어야산다#양식#스테이크#정대후문#혼밥#고기#함박#가성비
3,언니네반점,Num=a_nam_0146,#먹어야산다#중식#정대후문#사천식#혼밥#가성비#저렴#포장#회식
4,특별식당,Num=a_nam_0319,#먹어야산다#일식#돈부리#카레#맥주#고로케#제기동#고대사거리#혼밥#감성


key['key_words'] = ""

In [8]:
keywords = []

In [9]:
for i in range(465):
    tag = key['tag'][i]
    
    r = Rake()
    r.extract_keywords_from_text(tag)
    
    words = r.get_ranked_phrases()
    keywords.append(words)


In [10]:
key['key_words'] = keywords

In [12]:
key.head()

,name,code,tag,key_words
0,고른햇살,Num=a_nam_0002,#먹어야산다#분식#정대후문#포장#혼밥#가성비,"[혼밥, 포장, 정대후문, 분식, 먹어야산다, 가성비]"
1,영철버거,Num=a_nam_0041,#먹어야산다#이공계#경양식#버거#맥주#치킨#가성비#저렴#포장,"[포장, 치킨, 저렴, 이공계, 버거, 먹어야산다, 맥주, 경양식, 가성비]"
2,어흥식당,Num=a_nam_829,#먹어야산다#양식#스테이크#정대후문#혼밥#고기#함박#가성비,"[혼밥, 함박, 정대후문, 양식, 스테이크, 먹어야산다, 고기, 가성비]"
3,언니네반점,Num=a_nam_0146,#먹어야산다#중식#정대후문#사천식#혼밥#가성비#저렴#포장#회식,"[회식, 혼밥, 포장, 중식, 정대후문, 저렴, 사천식, 먹어야산다, 가성비]"
4,특별식당,Num=a_nam_0319,#먹어야산다#일식#돈부리#카레#맥주#고로케#제기동#고대사거리#혼밥#감성,"[혼밥, 카레, 제기동, 일식, 먹어야산다, 맥주, 돈부리, 고로케, 고대사거리, 감성]"


In [13]:
def dummy(doc):
    return doc

In [14]:
count = CountVectorizer(tokenizer=dummy, preprocessor=dummy)
count_matrix = count.fit_transform(key['key_words'])

count_matrix = count_matrix.toarray()

# 코사인 유사도
c_sim = cosine_similarity(count_matrix, count_matrix)

In [15]:
type(count_matrix)

numpy.ndarray

In [16]:
print(c_sim)

[[1.         0.40824829 0.57735027 ... 0.40824829 0.23570226 0.16666667]
 [0.40824829 1.         0.23570226 ... 0.33333333 0.19245009 0.13608276]
 [0.57735027 0.23570226 1.         ... 0.35355339 0.20412415 0.14433757]
 ...
 [0.40824829 0.33333333 0.35355339 ... 1.         0.57735027 0.40824829]
 [0.23570226 0.19245009 0.20412415 ... 0.57735027 1.         0.23570226]
 [0.16666667 0.13608276 0.14433757 ... 0.40824829 0.23570226 1.        ]]


In [17]:
print(c_sim.shape)

(465, 465)


In [18]:
c_sim = pd.DataFrame(data = c_sim, index = key['name'], columns = key['name'])
c_sim.head()

name,고른햇살,영철버거,어흥식당,언니네반점,특별식당,돈까스하우스,히포크라테스 스프,곰이네국밥,비나 레스토랑 정문,주유소,...,홍빠오,긴자 료코,자장면,원조족발보쌈,소고기 불 초밥,한식,피자,안암역 3출쪽 포장마차,벤베누토,오뚜기식당
name,,,,,,,,,,,,,,,,,,,,,
고른햇살,1.000000,0.408248,0.577350,0.680414,0.258199,0.721688,0.369274,0.462910,0.333333,0.433013,...,0.204124,0.204124,0.288675,0.235702,0.408248,0.408248,0.408248,0.408248,0.235702,0.166667
영철버거,0.408248,1.000000,0.235702,0.444444,0.210819,0.471405,0.201008,0.377964,0.272166,0.471405,...,0.166667,0.166667,0.235702,0.192450,0.333333,0.333333,0.333333,0.333333,0.192450,0.136083
어흥식당,0.577350,0.235702,1.000000,0.471405,0.223607,0.500000,0.213201,0.400892,0.144338,0.375000,...,0.176777,0.176777,0.250000,0.204124,0.353553,0.353553,0.353553,0.353553,0.204124,0.144338
언니네반점,0.680414,0.444444,0.471405,1.000000,0.210819,0.589256,0.301511,0.503953,0.408248,0.589256,...,0.166667,0.166667,0.235702,0.192450,0.333333,0.333333,0.333333,0.333333,0.192450,0.136083
특별식당,0.258199,0.210819,0.223607,0.210819,1.000000,0.223607,0.286039,0.358569,0.129099,0.111803,...,0.158114,0.316228,0.223607,0.365148,0.316228,0.316228,0.316228,0.316228,0.182574,0.129099


In [19]:
c_sim[c_sim.index == '영철버거'].values[0]

array([0.40824829, 1.        , 0.23570226, 0.44444444, 0.21081851,
       0.47140452, 0.20100756, 0.37796447, 0.27216553, 0.47140452,
       0.44444444, 0.37796447, 0.31622777, 0.44444444, 0.        ,
       0.33333333, 0.35355339, 0.4472136 , 0.        , 0.27216553,
       0.23570226, 0.21081851, 0.21081851, 0.47140452, 0.23570226,
       0.52704628, 0.50251891, 0.27216553, 0.44444444, 0.35355339,
       0.44444444, 0.11111111, 0.10050378, 0.27216553, 0.17817416,
       0.19245009, 0.13608276, 0.25197632, 0.25197632, 0.16666667,
       0.37796447, 0.27216553, 0.31622777, 0.31622777, 0.27216553,
       0.2981424 , 0.35355339, 0.42163702, 0.20100756, 0.40201513,
       0.23570226, 0.        , 0.52704628, 0.33333333, 0.22222222,
       0.47140452, 0.2981424 , 0.12598816, 0.10540926, 0.33333333,
       0.12598816, 0.22222222, 0.23570226, 0.27216553, 0.40824829,
       0.25197632, 0.66666667, 0.23570226, 0.23570226, 0.11785113,
       0.33333333, 0.2981424 , 0.31622777, 0.44444444, 0.27216

In [20]:
a = c_sim[c_sim.index == '영철버거'].values[0]
np.sort(a)[::-1][:5]

array([1.        , 0.66666667, 0.55555556, 0.55555556, 0.54433105])

In [21]:
c_sim.index[1]

'영철버거'

In [22]:
def recommendation(name, top):
    indices = c_sim[c_sim.index == name].values[0]
    indices = np.sort(indices)[::-1][:top*2]
    
    temp = []
    for k in range(len(indices)):
        idx_name = c_sim.index[indices[k] == c_sim[c_sim.index == name].values[0]]
        temp.append(idx_name[0])
    
    # 중복된 값 제거
    recommend = []
    for t in temp:
        if t in recommend:
            pass
        else:
            recommend.append(t)
    
    # 입력값과 동일한 항목은 제거
    recommend = recommend[1:top+1]
    return recommend
    

In [23]:
recommendation('고른햇살', 8)

['김밥천국 정대후문점', '절대분식', '엄마네', '돈까스하우스', '안암동김밥', '언니네반점', '이세돈까스', '이공김밥']

In [24]:
recommendation('매스플레이트', 4)

['정상호프', '어흥식당', '무르무르드구스토', '모이리타']

In [25]:
recommendation('베나레스', 5)

['오샬', '비나 레스토랑 정문', '고고인디안쿠진 1호점', '돼야지 고대본점', '고고인디안쿠진 2호점']